## Import Packages

In [2]:
import csv
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

## Extract Data

In [36]:
def findCommunities(filename):
    communities = []
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                if i[1] not in communities:
                    communities.append(i[1])
            j+=1
    return communities
def extractData(filename,communities,addstate = False,addcommunity = False):
    features = []
    xPositive=[]
    yPositive=[]
    xNegative=[]
    yNegative =[]
    with open(filename, 'r') as csvfile:
        filereader = csv.reader(csvfile)
        j = 0
        for i in filereader:
            if j > 0:
                data = i[3:len(i)-1]
                if addstate:
                    state = [0.0]*56
                    state[int(i[0])-1] = 1.0
                    data = data + state
                    features = features + ["state"+str(i) for i in range(56)]
                if addcommunity:
                    tmp = [0.0]*len(communities)
                    tmp[communities.index(i[1])] = 1.0
                    data = data + tmp
                    features = features + communities
                vect = np.array(data)
                #vect = np.array(i[3:len(i)-1])
                #x.append(vect.astype(np.float))
                if float(i[-1]) > 0.1:
                    xPositive.append(vect.astype(np.float))
                    yPositive.append(1)
                else:
                    xNegative.append(vect.astype(np.float))
                    yNegative.append(0)
            else:
                features = i[3:len(i)-1]
            j+=1
        """yPositive = np.array(yPositive)
        xPositive = np.array(xPositive)
        yNegative = np.array(yNegative)
        xNegative = np.array(xNegative)"""
        features = np.array(features)
        indxP = int(len(xPositive)*.6)
        indxN = int(len(xNegative)*.6)
        xTrain =np.array(xPositive[:indxP] + xNegative[:indxN])
        yTrain = np.array(yPositive[:indxP] + yNegative[:indxN])
        xTest = np.array(xPositive[indxP:] + xNegative[indxN:])
        yTest = np.array(yPositive[indxP:] + yNegative[indxN:])
        
        
            
            
        return xTrain,yTrain,xTest,yTest,features

## Calculate Label Percentage

In [37]:
def calcualtePercentage(labels):
    n = len(labels)
    sumTrue = sum(labels)*1.0
    sumFalse = (n - sumTrue)*1.0
    return sumTrue/n,sumFalse/n

## Create Model

In [69]:
def clfSVC(kernel = 'rbf'):
    return SVC(C=1.0, kernel= kernel, degree=3, gamma='auto', coef0=0.0,
               shrinking=True, probability=False, tol=0.001, cache_size=200, 
               class_weight=None, verbose=False, max_iter=-1,decision_function_shape=None, random_state=None)

## Evaluation

In [70]:
def confusionMatrix(true,predicted,labels = [1,0]):
    return confusion_matrix(true,predicted,labels)

def find_confusion_matrix(actual,predicted,clabels):
    cm= []
    for i in clabels:
        tmp =[0]*len(clabels)
        
        for j in range(len(actual)):
            if actual[j]== i and actual[j] == predicted[j]:
                tmp[clabels.index(i)] += 1
            elif actual[j]== i and actual[j] != predicted[j]:
                tmp[clabels.index(predicted[j])] += 1
        cm.append(tmp)
    return np.array(cm)
def find_accuracy(matrix):
    return np.trace(matrix)*1.0/np.sum(matrix)
def find_precision(matrix):
    pres = []
    x = np.sum(matrix,axis=0)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                pres.append(matrix[i][j]*1.0/x[i])
    return pres
def find_recall(matrix):
    rec = []
    x = np.sum(matrix,axis=1)
    for i in range(len(matrix)):
        for j in range(len(matrix)):
            if i == j:
                rec.append(matrix[i][j]*1.0/x[i])
    return rec
def evaluation(acutal,predicted,clabels=[1,0]):
    confmatrix = find_confusion_matrix(acutal,predicted,clabels)
    print ("Confusion Matrix")
    print (confmatrix)
    accuracy = find_accuracy(confmatrix)
    print ("Accuracy", accuracy)
    precision = find_precision(confmatrix)
    print ("Precision", precision)
    recall = find_recall(confmatrix)
    print ("Recall", recall)

## Cross Validation

In [71]:
def do_cross_validation(X, y,clf, n_folds=5):
    cv = KFold(len(y), n_folds, shuffle=True)
    accuracies = []
    i = 1
    for train_ind, test_ind in cv: 
        clf.fit(X[train_ind], y[train_ind])
        predictions = clf.predict(X[test_ind])
        print ("Fold %d Evaluation" %i)
        i +=1
        evaluation(y[test_ind],predictions)
        print ("\n")
        accuracies.append(accuracy_score(y[test_ind], predictions))
    avg = np.mean(accuracies)
    print ("The average Accuracy is ", avg)

### Read Data

In [72]:
filename = 'C:\\Users\\ar1\\Documents\ML\\Project\\Crime Prediction Data(1)\\Crime Prediction Data\\communities-crime-clean.csv'
distintCommunities = findCommunities(filename)
XTrain,YTrain,XTest,YTest,Features = extractData(filename, distintCommunities)

### Percentage of Labels

In [73]:
print ("Train Percentage",calcualtePercentage(YTrain))
print ("Test Percentage",calcualtePercentage(YTest))

Train Percentage (0.62761506276150625, 0.3723849372384937)
Test Percentage (0.62656641604010022, 0.37343358395989973)


### Default SVC Model

In [88]:
model= clfSVC()
model.fit(XTrain,YTrain)
predictedY=model.predict(XTest)
evaluation(YTest,predictedY)

Confusion Matrix
[[436  64]
 [ 79 219]]
Accuracy 0.820802005013
Precision [0.84660194174757286, 0.77385159010600701]
Recall [0.872, 0.7348993288590604]


### 10 Fold Cross Validation on Default SVC Model

In [75]:
do_cross_validation(XTrain,YTrain,model,10)

Fold 1 Evaluation
Confusion Matrix
[[66  9]
 [15 30]]
Accuracy 0.8
Precision [0.81481481481481477, 0.76923076923076927]
Recall [0.88, 0.66666666666666663]


Fold 2 Evaluation
Confusion Matrix
[[71 13]
 [ 4 32]]
Accuracy 0.858333333333
Precision [0.94666666666666666, 0.71111111111111114]
Recall [0.84523809523809523, 0.88888888888888884]


Fold 3 Evaluation
Confusion Matrix
[[78  8]
 [ 9 25]]
Accuracy 0.858333333333
Precision [0.89655172413793105, 0.75757575757575757]
Recall [0.90697674418604646, 0.73529411764705888]


Fold 4 Evaluation
Confusion Matrix
[[65  9]
 [16 30]]
Accuracy 0.791666666667
Precision [0.80246913580246915, 0.76923076923076927]
Recall [0.8783783783783784, 0.65217391304347827]


Fold 5 Evaluation
Confusion Matrix
[[62  7]
 [16 35]]
Accuracy 0.808333333333
Precision [0.79487179487179482, 0.83333333333333337]
Recall [0.89855072463768115, 0.68627450980392157]


Fold 6 Evaluation
Confusion Matrix
[[62  8]
 [ 9 40]]
Accuracy 0.857142857143
Precision [0.87323943661971826, 0.

### Evaluation for Different Kernels

In [76]:
kernel = ['linear','poly', 'rbf', 'sigmoid',]
modelList =[]
for i in kernel:
    model= clfSVC(kernel = i)
    model.fit(XTrain,YTrain)
    modelList.append(model)
    predictedY=model.predict(XTest)
    print("\n"+i)
    evaluation(YTest,predictedY)
    


linear
Confusion Matrix
[[417  83]
 [ 57 241]]
Accuracy 0.824561403509
Precision [0.879746835443038, 0.74382716049382713]
Recall [0.83399999999999996, 0.8087248322147651]

poly
Confusion Matrix
[[500   0]
 [292   6]]
Accuracy 0.634085213033
Precision [0.63131313131313127, 1.0]
Recall [1.0, 0.020134228187919462]

rbf
Confusion Matrix
[[436  64]
 [ 79 219]]
Accuracy 0.820802005013
Precision [0.84660194174757286, 0.77385159010600701]
Recall [0.872, 0.7348993288590604]

sigmoid
Confusion Matrix
[[444  56]
 [ 99 199]]
Accuracy 0.805764411028
Precision [0.81767955801104975, 0.7803921568627451]
Recall [0.88800000000000001, 0.66778523489932884]


### Top 10 important features in Linear model

In [86]:
coef = modelList[0].coef_
topindx = np.argsort(coef)[0][::-1][:10]
print (Features[topindx])


['PctUnemployed' 'racePctHisp' 'PctImmigRecent' 'PctIlleg' 'agePct12t21'
 'MalePctDivorce' 'racepctblack' 'PctImmigRec10' 'TotalPctDiv'
 'PersPerOccupHous']
